In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import warnings
from datetime import datetime, timedelta
import math
from operator import attrgetter # Added for cohort analysis lambda function

warnings.filterwarnings("ignore", category=RuntimeWarning)

In [ ]:
# Read the CSV files (update paths as needed)
file1 = "/kaggle/input/crm-data-analysis/sales_order_01.04.23-01.04.24.78558153.csv"
file2 = "/kaggle/input/crm-data-analysis/sales_order_01.04.24-01.04.25.81099654.csv"

# Load the data into DataFrames
df1 = pd.read_csv(file1, encoding='utf-8', low_memory=False)
df2 = pd.read_csv(file2, encoding='utf-8', low_memory=False)

print(f"✅ Dataset 1 (2023-2024): {df1.shape}")
print(f"✅ Dataset 2 (2024-2025): {df2.shape}")

# Merge the DataFrames
merged_df = pd.concat([df1, df2], ignore_index=True, sort=False)
print(f"✅ Merged dataset shape: {merged_df.shape}")

In [ ]:
df1.info()

In [ ]:
df2.info()

In [ ]:
# Merge the DataFrames
merged_df = pd.concat([df1, df2], ignore_index=True, sort=False)
print(f"✅ Merged dataset shape: {merged_df.shape}")

merged_df.head(3)

In [ ]:
merged_df.info()

In [ ]:
merged_df2 = merged_df
merged_df2.head(3)

In [ ]:
# Step 1: Keep only necessary columns
merged_df2 = merged_df[['Customer Email', 'Purchase Date']]

# Step 2: Convert 'Purchase Date' to datetime
merged_df2['Purchase Date'] = pd.to_datetime(merged_df2['Purchase Date'], errors='coerce')

# Step 3: Create 'Year' and 'Month-Year' columns
merged_df2['Year'] = merged_df2['Purchase Date'].dt.year
merged_df2['Month-Year'] = merged_df2['Purchase Date'].dt.to_period('M').astype(str)

# Step 4: Remove null emails or dates
merged_df2 = merged_df2.dropna(subset=['Customer Email', 'Purchase Date'])

# Step 5: Calculate Retention Rate YoY
retention = {}
years = sorted(merged_df2['Year'].dropna().unique())

for i in range(len(years) - 1):
    year_curr = years[i]
    year_next = years[i + 1]

    users_curr = set(merged_df2[merged_df2['Year'] == year_curr]['Customer Email'].unique())
    users_next = set(merged_df2[merged_df2['Year'] == year_next]['Customer Email'].unique())

    retained_users = users_curr & users_next
    retention_rate = len(retained_users) / len(users_curr) if users_curr else 0

    retention[f"{year_curr} → {year_next}"] = round(retention_rate * 100, 2)

# Step 6: Calculate Repeat Purchase Rate (1st to 2nd)
repeat_purchase = {}
for year in years:
    year_data = merged_df2[merged_df2['Year'] == year]
    purchase_counts = year_data.groupby('Customer Email').size()
    
    one_time = purchase_counts[purchase_counts == 1].count()
    repeaters = purchase_counts[purchase_counts >= 2].count()

    total_customers = one_time + repeaters
    repeat_rate = repeaters / total_customers if total_customers else 0

    repeat_purchase[str(year)] = round(repeat_rate * 100, 2)

In [ ]:
# Output results
print("📊 Year-over-Year Retention Rate:")
for k, v in retention.items():
    print(f"{k}: {v}%")

print("\n🔁 Repeat Purchase Rate:")
for k, v in repeat_purchase.items():
    print(f"{k}: {v}%")

In [ ]:
# Load retention data
retention_df = pd.read_csv('/kaggle/input/crm-data-analysis/Customer Retention Rate Mehedi.csv')
print(f"✅ Retention data loaded: {retention_df.shape}")

retention_df.info()

In [ ]:
retention_df.head(3)

## Monthly & Yearly Retention Repurchase Churn (Customer Retention)

In [ ]:
import pandas as pd

# Load data
retention_df = pd.read_csv('/kaggle/input/crm-data-analysis/Customer Retention Rate Mehedi.csv')
print(f"✅ Retention data loaded: {retention_df.shape}")

# Keep only necessary columns
retention_df = retention_df[['EMAIL', 'FIRST_ORDER_DATE', 'LAST_ORDER_DATE']].dropna()

# Convert date columns to datetime
retention_df['FIRST_ORDER_DATE'] = pd.to_datetime(retention_df['FIRST_ORDER_DATE'], errors='coerce')
retention_df['LAST_ORDER_DATE'] = pd.to_datetime(retention_df['LAST_ORDER_DATE'], errors='coerce')

# Drop rows with invalid dates
retention_df = retention_df.dropna(subset=['FIRST_ORDER_DATE', 'LAST_ORDER_DATE'])

# Create Year and Month-Year columns
retention_df['FIRST_YEAR'] = retention_df['FIRST_ORDER_DATE'].dt.year
retention_df['LAST_YEAR'] = retention_df['LAST_ORDER_DATE'].dt.year

retention_df['FIRST_MONTH_YEAR'] = retention_df['FIRST_ORDER_DATE'].dt.to_period('M').astype(str)
retention_df['LAST_MONTH_YEAR'] = retention_df['LAST_ORDER_DATE'].dt.to_period('M').astype(str)

# Unique customers
customers_df = retention_df.drop_duplicates(subset='EMAIL')

# Overall Repeat Purchase Rate (for reference)
repeat_df = customers_df[customers_df['FIRST_ORDER_DATE'] != customers_df['LAST_ORDER_DATE']]
overall_repeat_purchase_rate = len(repeat_df) / len(customers_df)
print(f"🔁 Overall Repeat Purchase Rate: {overall_repeat_purchase_rate:.2%}")

# ======================= #
# 📅 Year-over-Year (YoY) #
# ======================= #
yoy_metrics = []

years = sorted(customers_df['FIRST_YEAR'].dropna().unique())
for i in range(1, len(years)):
    prev_year = years[i - 1]
    current_year = years[i]

    prev_customers = set(customers_df[customers_df['FIRST_YEAR'] <= prev_year]['EMAIL'])
    current_customers = set(customers_df[customers_df['LAST_YEAR'] == current_year]['EMAIL'])

    retained = current_customers.intersection(prev_customers)
    retention_rate = len(retained) / len(prev_customers) if prev_customers else 0
    churn_rate = 1 - retention_rate

    # Calculate dynamic Repeat Purchase Rate for YoY period
    repeat_customers = customers_df[
        (customers_df['FIRST_YEAR'] <= prev_year) &
        (customers_df['LAST_YEAR'] == current_year) &
        (customers_df['FIRST_ORDER_DATE'] != customers_df['LAST_ORDER_DATE'])
    ]
    year_repeat_purchase_rate = len(repeat_customers) / len(prev_customers) if prev_customers else 0

    yoy_metrics.append({
        'Year': current_year,
        'Retention Rate': round(retention_rate, 6),
        'Churn Rate': round(churn_rate, 6),
        'Repeat Purchase Rate': round(year_repeat_purchase_rate, 6)
    })

# ============================ #
# 📆 Month-over-Month (MoM)    #
# ============================ #
mom_metrics = []

months = sorted(customers_df['FIRST_MONTH_YEAR'].dropna().unique())
for i in range(1, len(months)):
    prev_month = months[i - 1]
    current_month = months[i]

    prev_customers = set(customers_df[customers_df['FIRST_MONTH_YEAR'] <= prev_month]['EMAIL'])
    current_customers = set(customers_df[customers_df['LAST_MONTH_YEAR'] == current_month]['EMAIL'])

    retained = current_customers.intersection(prev_customers)
    retention_rate = len(retained) / len(prev_customers) if prev_customers else 0
    churn_rate = 1 - retention_rate

    # Calculate dynamic Repeat Purchase Rate for MoM period
    repeat_customers = customers_df[
        (customers_df['FIRST_MONTH_YEAR'] <= prev_month) &
        (customers_df['LAST_MONTH_YEAR'] == current_month) &
        (customers_df['FIRST_ORDER_DATE'] != customers_df['LAST_ORDER_DATE'])
    ]
    month_repeat_purchase_rate = len(repeat_customers) / len(prev_customers) if prev_customers else 0

    mom_metrics.append({
        'Month': current_month,
        'Retention Rate': round(retention_rate, 6),
        'Churn Rate': round(churn_rate, 6),
        'Repeat Purchase Rate': round(month_repeat_purchase_rate, 6)
    })

In [ ]:
# You can now print or save yoy_metrics and mom_metrics as needed
print("✅ YoY metrics sample:")
yoy_metrics[:4]

In [ ]:
print("✅ MoM metrics sample:")
mom_metrics[:4]

In [ ]:
# Save YoY
yoy_df = pd.DataFrame(yoy_metrics)
yoy_df.to_csv('yearly_retention_repurchase_churn_customer_retention.csv', index=False)
print("📁 YoY metrics saved to 'yearly_retention_repurchase_churn_customer_retention.csv'")

# Save MoM
mom_df = pd.DataFrame(mom_metrics)
mom_df.to_csv('monthly_retention_repurchase_churn_customer_retention.csv', index=False)
print("📁 MoM metrics saved to 'monthly_retention_repurchase_churn_customer_retention.csv'")

In [ ]:
# 3. DATA CLEANING AND PREPROCESSING
print("\n🧹 Step 3: Data cleaning and preprocessing...")

# Clean merged sales data
def clean_sales_data(df):
    """Clean and preprocess sales data"""
    df_clean = df.copy()
    
    # Convert Purchase Date to datetime
    df_clean['Purchase Date'] = pd.to_datetime(df_clean['Purchase Date'], errors='coerce')
    
    # Extract date components
    df_clean['Year'] = df_clean['Purchase Date'].dt.year
    df_clean['Month'] = df_clean['Purchase Date'].dt.month
    df_clean['Quarter'] = df_clean['Purchase Date'].dt.quarter
    df_clean['Day_of_Week'] = df_clean['Purchase Date'].dt.day_name()
    df_clean['Month_Year'] = df_clean['Purchase Date'].dt.to_period('M').astype(str)
    
    # Clean email addresses
    df_clean['Customer Email'] = df_clean['Customer Email'].str.lower().str.strip()
    
    # Handle missing values in critical columns
    df_clean['Customer Group'] = df_clean['Customer Group'].fillna('Unknown')
    df_clean['Status'] = df_clean['Status'].fillna('Unknown')
    
    # Create customer segments based on order value
    df_clean['Order_Value_Segment'] = pd.cut(
        df_clean['Grand Total (Base)'], 
        bins=[-np.inf, 50, 100, 200, np.inf], 
        labels=['Low (0-50)', 'Medium (50-100)', 'High (100-200)', 'Premium (200+)']
    )
    
    # Remove rows with invalid purchase dates or zero/negative amounts
    df_clean = df_clean.dropna(subset=['Purchase Date'])
    df_clean = df_clean[df_clean['Grand Total (Base)'] > 0]
    
    return df_clean

# Clean retention data
def clean_retention_data(df):
    """Clean and preprocess retention data"""
    df_clean = df.copy()
    
    # Clean email addresses
    df_clean['EMAIL'] = df_clean['EMAIL'].str.lower().str.strip()
    
    # Convert date columns
    date_columns = ['FIRST_ORDER_DATE', 'LAST_ORDER_DATE', 'ACCOUNT_CREATION_DATE']
    for col in date_columns:
        if col in df_clean.columns:
            df_clean[col] = pd.to_datetime(df_clean[col], errors='coerce')
    
    # Remove rows with missing email or critical data
    df_clean = df_clean.dropna(subset=['EMAIL'])
    df_clean = df_clean[df_clean['EMAIL'].str.contains('@', na=False)]
    
    # Fill missing values
    df_clean['GENDER'] = df_clean['GENDER'].fillna('Unknown')
    df_clean['REGION'] = df_clean['REGION'].fillna('Unknown')
    df_clean['LOYALTY_TIER'] = df_clean['LOYALTY_TIER'].fillna('None')
    df_clean['TOTAL_ORDER_AMOUNT'] = df_clean['TOTAL_ORDER_AMOUNT'].fillna(0)
    
    return df_clean

# Apply cleaning
sales_clean = clean_sales_data(merged_df)

In [ ]:
retention_clean = clean_retention_data(retention_df)

In [ ]:
print(f"✅ Sales data cleaned: {sales_clean.shape}")
print(f"✅ Retention data cleaned: {retention_clean.shape}")

In [ ]:
print(f"✅ Sales data cleaned: {sales_clean.shape}")
print(f"✅ Retention data cleaned: {retention_clean.shape}")

In [ ]:
sales_clean.info()

In [ ]:
sales_clean.head(3)

## Monthly & Yearly Retention Repurchase Churn (Sales Order)

In [ ]:
import pandas as pd

# Step 1: Ensure necessary columns are present and valid
sales_df = sales_clean[['Customer Email', 'Purchase Date', 'Year']].copy()

# Step 2: Drop rows with null values in essential columns
sales_df.dropna(subset=['Customer Email', 'Purchase Date'], inplace=True)

# Step 3: Calculate Year-over-Year Retention Rate
retention = {}
years = sorted(sales_df['Year'].dropna().unique())

for i in range(len(years) - 1):
    year_curr = years[i]
    year_next = years[i + 1]

    users_curr = set(sales_df[sales_df['Year'] == year_curr]['Customer Email'].unique())
    users_next = set(sales_df[sales_df['Year'] == year_next]['Customer Email'].unique())

    retained_users = users_curr & users_next
    retention_rate = len(retained_users) / len(users_curr) if users_curr else 0

    retention[f"{year_curr} → {year_next}"] = round(retention_rate * 100, 2)

# Step 4: Calculate Repeat Purchase Rate (1st to 2nd)
repeat_purchase = {}
for year in years:
    year_data = sales_df[sales_df['Year'] == year]
    purchase_counts = year_data.groupby('Customer Email').size()
    
    one_time = purchase_counts[purchase_counts == 1].count()
    repeaters = purchase_counts[purchase_counts >= 2].count()

    total_customers = one_time + repeaters
    repeat_rate = repeaters / total_customers if total_customers else 0

    repeat_purchase[str(year)] = round(repeat_rate * 100, 2)

# Step 5: Print Results
print("📊 Year-over-Year Retention Rate:")
for k, v in retention.items():
    print(f"{k}: {v}%")

print("\n🔁 Repeat Purchase Rate:")
for k, v in repeat_purchase.items():
    print(f"{k}: {v}%")


In [ ]:
sales_df.head()

In [ ]:
import pandas as pd

# Assuming 'sales_clean' is already loaded as a DataFrame

# Step 1: Keep only necessary columns
cols_to_keep = ['Customer Email', 'Purchase Date']
sales = sales_clean[cols_to_keep].copy()

# Step 2: Create Month_Year and Year columns if not already
sales['Year'] = sales['Purchase Date'].dt.year
sales['Month_Year'] = sales['Purchase Date'].dt.to_period('M').astype(str)

# Step 3: Sort by Purchase Date
sales = sales.sort_values(by=['Customer Email', 'Purchase Date'])

# Step 4: Calculate total purchases per customer
purchase_counts = sales.groupby('Customer Email')['Purchase Date'].count().reset_index()
purchase_counts.columns = ['Customer Email', 'Purchase_Count']

# Merge purchase count back to sales
sales = sales.merge(purchase_counts, on='Customer Email')

# Step 5: First Purchase Date
first_purchase = sales.groupby('Customer Email')['Purchase Date'].min().reset_index()
first_purchase.columns = ['Customer Email', 'First_Purchase_Date']

# Merge to get first purchase month/year
sales = sales.merge(first_purchase, on='Customer Email')
sales['First_Purchase_Month'] = sales['First_Purchase_Date'].dt.to_period('M').astype(str)
sales['First_Purchase_Year'] = sales['First_Purchase_Date'].dt.year

# Step 6: Retention Rate (MoM and YoY)
monthly_customers = sales.groupby('Month_Year')['Customer Email'].nunique().reset_index()
monthly_customers.columns = ['Month_Year', 'Active_Customers']

monthly_customers['Prev_Month_Customers'] = monthly_customers['Active_Customers'].shift(1)
monthly_customers['Retention_Rate'] = (monthly_customers['Active_Customers'] / monthly_customers['Prev_Month_Customers']) * 100

yearly_customers = sales.groupby('Year')['Customer Email'].nunique().reset_index()
yearly_customers.columns = ['Year', 'Active_Customers']

yearly_customers['Prev_Year_Customers'] = yearly_customers['Active_Customers'].shift(1)
yearly_customers['Retention_Rate'] = (yearly_customers['Active_Customers'] / yearly_customers['Prev_Year_Customers']) * 100

# Step 7: Repeat Purchase Rate
# Monthly
monthly_repeat = sales[sales['Purchase_Count'] > 1].groupby('Month_Year')['Customer Email'].nunique().reset_index()
monthly_repeat.columns = ['Month_Year', 'Repeat_Customers']
monthly_repeat = monthly_repeat.merge(monthly_customers[['Month_Year', 'Active_Customers']], on='Month_Year', how='left')
monthly_repeat['Repeat_Purchase_Rate'] = (monthly_repeat['Repeat_Customers'] / monthly_repeat['Active_Customers']) * 100

# Yearly
yearly_repeat = sales[sales['Purchase_Count'] > 1].groupby('Year')['Customer Email'].nunique().reset_index()
yearly_repeat.columns = ['Year', 'Repeat_Customers']
yearly_repeat = yearly_repeat.merge(yearly_customers[['Year', 'Active_Customers']], on='Year', how='left')
yearly_repeat['Repeat_Purchase_Rate'] = (yearly_repeat['Repeat_Customers'] / yearly_repeat['Active_Customers']) * 100

# Step 8: Churn Rate (100 - Retention)
monthly_customers['Churn_Rate'] = 100 - monthly_customers['Retention_Rate']
yearly_customers['Churn_Rate'] = 100 - yearly_customers['Retention_Rate']

In [ ]:
# Step 9: Save as CSVs
monthly_output = monthly_customers.merge(monthly_repeat[['Month_Year', 'Repeat_Purchase_Rate']], on='Month_Year', how='left')
yearly_output = yearly_customers.merge(yearly_repeat[['Year', 'Repeat_Purchase_Rate']], on='Year', how='left')

monthly_output.to_csv('monthly_retention_repurchase_churn.csv', index=False)
yearly_output.to_csv('yearly_retention_repurchase_churn.csv', index=False)

In [ ]:
monthly_output.head(3) 

In [ ]:
yearly_output.head(3)

## GAP between purchases

In [ ]:
import pandas as pd

# Step 1: Keep necessary columns
sales_df = sales_clean[['Customer Email', 'Purchase Date']].copy()

# Step 2: Convert 'Purchase Date' to datetime and drop nulls
sales_df['Purchase Date'] = pd.to_datetime(sales_df['Purchase Date'], errors='coerce')
sales_df.dropna(subset=['Customer Email', 'Purchase Date'], inplace=True)

# Step 3: Sort data by customer and purchase date
sales_df.sort_values(by=['Customer Email', 'Purchase Date'], inplace=True)

# Step 4: For each customer, calculate the difference between 2nd and 1st purchase
def get_1st_2nd_gap(df):
    if len(df) < 2:
        return None  # Not enough purchases
    return (df.iloc[1] - df.iloc[0]).days

gap_df = sales_df.groupby('Customer Email')['Purchase Date'].apply(get_1st_2nd_gap).dropna().reset_index()
gap_df.columns = ['Customer Email', 'Days Between 1st and 2nd Purchase']

# Step 5: Display results
print(gap_df.head())

# Optional: Save to CSV
gap_df.to_csv('1st_to_2nd_purchase_gap.csv', index=False)


In [ ]:
average_gap = gap_df['Days Between 1st and 2nd Purchase'].mean()
print(f"\n📅 Average Days Between 1st and 2nd Purchase: {round(average_gap, 2)} days")

In [ ]:
# 4. GENERATE ANALYTICS DATASETS
print("\n📊 Step 4: Generating analytics datasets...")

# 4.1 Customer Analytics Dataset
def create_customer_analytics(sales_df, retention_df):
    """Create comprehensive customer analytics dataset"""
    
    # Customer aggregations from sales data
    customer_sales = sales_df.groupby('Customer Email').agg({
        'ID': 'count',  # Total orders
        'Grand Total (Base)': ['sum', 'mean', 'std'],
        'Purchase Date': ['min', 'max'],
        'Discount Amount': 'sum',
        'Status': lambda x: (x == 'complete').sum(),  # Completed orders
        'Customer Group': 'first'
    }).round(2)
    
    # Flatten column names
    customer_sales.columns = [
        'Total_Orders', 'Total_Revenue', 'Average_Order_Value', 'Order_Value_StdDev',
        'First_Purchase_Date', 'Last_Purchase_Date', 'Total_Discount_Used',
        'Completed_Orders', 'Customer_Group'
    ]
    
    # Calculate additional metrics
    customer_sales['Days_Between_First_Last'] = (
        customer_sales['Last_Purchase_Date'] - customer_sales['First_Purchase_Date']
    ).dt.days
    
    customer_sales['Purchase_Frequency_Days'] = (
        customer_sales['Days_Between_First_Last'] / customer_sales['Total_Orders'].clip(lower=1)
    ).round(2)
    
    customer_sales['Repeat_Customer'] = customer_sales['Total_Orders'] > 1
    customer_sales['Customer_Lifetime_Value'] = customer_sales['Total_Revenue']
    
    # Customer segments
    customer_sales['CLV_Segment'] = pd.cut(
        customer_sales['Customer_Lifetime_Value'],
        bins=[-np.inf, 100, 300, 500, np.inf],
        labels=['Low', 'Medium', 'High', 'VIP']
    )
    
    # Reset index to make email a column
    customer_sales = customer_sales.reset_index()
    customer_sales.rename(columns={'Customer Email': 'EMAIL'}, inplace=True)
    
    # Merge with retention data
    customer_analytics = customer_sales.merge(
        retention_df[['EMAIL', 'GENDER', 'REGION', 'LOYALTY_TIER', 'NEWSLETTER_FREQUENCY']],
        on='EMAIL', how='left'
    )
    
    return customer_analytics

# 4.2 Cohort Analysis Dataset
def create_cohort_analysis(sales_df):
    """Create cohort analysis dataset"""
    
    # Define cohorts based on first purchase month
    customer_cohorts = sales_df.groupby('Customer Email')['Purchase Date'].min().reset_index()
    customer_cohorts.columns = ['Customer Email', 'Cohort_Month']
    customer_cohorts['Cohort_Month'] = customer_cohorts['Cohort_Month'].dt.to_period('M')
    
    # Merge back with sales data
    sales_with_cohorts = sales_df.merge(customer_cohorts, on='Customer Email')
    sales_with_cohorts['Period_Number'] = (
        sales_with_cohorts['Purchase Date'].dt.to_period('M') - 
        sales_with_cohorts['Cohort_Month']
    ).apply(attrgetter('n'))
    
    # Create cohort table
    cohort_data = sales_with_cohorts.groupby(['Cohort_Month', 'Period_Number'])['Customer Email'].nunique().reset_index()
    cohort_data.columns = ['Cohort_Month', 'Period_Number', 'Customers']
    
    # Calculate cohort sizes
    cohort_sizes = customer_cohorts.groupby('Cohort_Month')['Customer Email'].nunique()
    cohort_data = cohort_data.merge(cohort_sizes.reset_index().rename(columns={'Customer Email': 'Cohort_Size'}), on='Cohort_Month')
    
    # Calculate retention rates
    cohort_data['Retention_Rate'] = (cohort_data['Customers'] / cohort_data['Cohort_Size'] * 100).round(2)
    
    return cohort_data

# Fix import for cohort analysis
from operator import attrgetter

# 4.3 Monthly Performance Dataset
def create_monthly_performance(sales_df):
    """Create monthly performance metrics"""
    
    monthly_metrics = sales_df.groupby('Month_Year').agg({
        'ID': 'count',
        'Customer Email': 'nunique',
        'Grand Total (Base)': ['sum', 'mean'],
        'Discount Amount': 'sum',
        'Status': lambda x: (x == 'complete').sum()
    }).round(2)
    
    monthly_metrics.columns = [
        'Total_Orders', 'Unique_Customers', 'Total_Revenue', 
        'Average_Order_Value', 'Total_Discounts', 'Completed_Orders'
    ]
    
    # Calculate additional metrics
    monthly_metrics['Completion_Rate'] = (
        monthly_metrics['Completed_Orders'] / monthly_metrics['Total_Orders'] * 100
    ).round(2)
    
    monthly_metrics['Revenue_Per_Customer'] = (
        monthly_metrics['Total_Revenue'] / monthly_metrics['Unique_Customers']
    ).round(2)
    
    return monthly_metrics.reset_index()

In [ ]:
# 4.4 Product Performance Dataset (if product data available)
def create_product_performance(sales_df):
    """Create product performance metrics"""
    
    # Group by customer group as proxy for product categories
    product_metrics = sales_df.groupby(['Customer Group', 'Month_Year']).agg({
        'ID': 'count',
        'Grand Total (Base)': ['sum', 'mean'],
        'Customer Email': 'nunique'
    }).round(2)
    
    product_metrics.columns = ['Orders', 'Revenue', 'AOV', 'Customers']
    
    return product_metrics.reset_index()

In [ ]:
# Generate all datasets
print("🔄 Generating customer analytics...")
customer_analytics = create_customer_analytics(sales_clean, retention_clean)

customer_analytics.head(3)

In [ ]:
print(f"Shape: {customer_analytics.shape}")

#### Generating cohort analysis

In [ ]:
print("🔄 Generating cohort analysis...")
cohort_analysis = create_cohort_analysis(sales_clean)
print(f"Shape: {cohort_analysis.shape}")
cohort_analysis.head(3)

In [ ]:
print("🔄 Generating monthly performance...")
monthly_performance = create_monthly_performance(sales_clean)

print(f"Shape: {monthly_performance.shape}")
monthly_performance.head(3)

In [ ]:
print("🔄 Generating product performance...")
product_performance = create_product_performance(sales_clean)

print(f"Shape: {product_performance.shape}")
product_performance.head(3)

In [ ]:
# 5. GENERATE KPI SUMMARY DATASET
print("\n📈 Step 5: Generating KPI summary...")

def create_kpi_summary(sales_df, customer_analytics):
    """Create KPI summary for dashboard"""
    
    total_customers = customer_analytics['EMAIL'].nunique()
    repeat_customers = customer_analytics['Repeat_Customer'].sum()
    total_revenue = sales_df['Grand Total (Base)'].sum()
    total_orders = len(sales_df)
    
    # Calculate retention rate
    retention_rate = (repeat_customers / total_customers * 100) if total_customers > 0 else 0
    
    # Calculate average time between purchases for repeat customers
    repeat_customer_data = customer_analytics[customer_analytics['Repeat_Customer']]
    avg_time_between_purchases = repeat_customer_data['Purchase_Frequency_Days'].mean()
    
    # Churn rate (customers who haven't purchased in last 90 days)
    cutoff_date = sales_df['Purchase Date'].max() - timedelta(days=90)
    active_customers = sales_df[sales_df['Purchase Date'] >= cutoff_date]['Customer Email'].nunique()
    churn_rate = ((total_customers - active_customers) / total_customers * 100) if total_customers > 0 else 0
    
    kpi_data = {
        'Metric': [
            'Total Customers', 'Repeat Customers', 'Customer Retention Rate (%)',
            'Customer Churn Rate (%)', 'Total Revenue', 'Total Orders', 
            'Average Order Value', 'Avg Time Between Purchases (Days)',
            'Repeat Purchase Rate (%)'
        ],
        'Value': [
            total_customers, repeat_customers, round(retention_rate, 2),
            round(churn_rate, 2), round(total_revenue, 2), total_orders,
            round(total_revenue / total_orders, 2), round(avg_time_between_purchases, 2),
            round((repeat_customers / total_customers * 100), 2)
        ]
    }
    
    return pd.DataFrame(kpi_data)

In [ ]:
kpi_summary = create_kpi_summary(sales_clean, customer_analytics)
print(f"Shape: {kpi_summary.shape}")
kpi_summary.head(3)

In [ ]:
# 6. SAVE ALL DATASETS
print("\n💾 Step 6: Saving processed datasets...")

datasets = {
    'merged_sales_clean.csv': sales_clean,
    'customer_analytics.csv': customer_analytics,
    'cohort_analysis.csv': cohort_analysis,
    'monthly_performance.csv': monthly_performance,
    'product_performance.csv': product_performance,
    'kpi_summary.csv': kpi_summary,
    'retention_data_clean.csv': retention_clean
}

for filename, dataset in datasets.items():
    dataset.to_csv(filename, index=False)
    print(f"✅ Saved: {filename} ({dataset.shape})")

# 7. GENERATE DATA DICTIONARY
print("\n📋 Step 7: Generating data dictionary...")

data_dictionary = {
    'File': [],
    'Column': [],
    'Description': [],
    'Data_Type': []
}

# Add descriptions for key datasets
file_descriptions = {
    'customer_analytics.csv': {
        'EMAIL': 'Customer email address (primary key)',
        'Total_Orders': 'Total number of orders placed by customer',
        'Total_Revenue': 'Total amount spent by customer',
        'Average_Order_Value': 'Average value per order',
        'Customer_Lifetime_Value': 'Total revenue from customer',
        'Repeat_Customer': 'Boolean - whether customer made repeat purchases',
        'CLV_Segment': 'Customer value segment (Low/Medium/High/VIP)'
    },
    'cohort_analysis.csv': {
        'Cohort_Month': 'Month when customer made first purchase',
        'Period_Number': 'Number of months after first purchase',
        'Customers': 'Number of customers active in this period',
        'Retention_Rate': 'Percentage of cohort still active'
    },
    'monthly_performance.csv': {
        'Month_Year': 'Month-Year period',
        'Total_Orders': 'Number of orders in month',
        'Total_Revenue': 'Revenue generated in month',
        'Unique_Customers': 'Number of unique customers in month',
        'Completion_Rate': 'Percentage of orders completed'
    }
}

for file, columns in file_descriptions.items():
    for col, desc in columns.items():
        data_dictionary['File'].append(file)
        data_dictionary['Column'].append(col)
        data_dictionary['Description'].append(desc)
        data_dictionary['Data_Type'].append('Various')

pd.DataFrame(data_dictionary).to_csv('data_dictionary.csv', index=False)

print("\n🎉 Data processing completed successfully!")
print("="*50)
print("📁 Generated files for Looker Studio:")
for filename in datasets.keys():
    print(f"   • {filename}")
print("   • data_dictionary.csv")

print("\n📊 Key Statistics:")
print(f"   • Total Orders: {len(sales_clean):,}")
print(f"   • Total Customers: {sales_clean['Customer Email'].nunique():,}")
print(f"   • Date Range: {sales_clean['Purchase Date'].min().strftime('%Y-%m-%d')} to {sales_clean['Purchase Date'].max().strftime('%Y-%m-%d')}")
print(f"   • Total Revenue: ${sales_clean['Grand Total (Base)'].sum():,.2f}")

print("\n🚀 Ready for Looker Studio import!")
print("Upload these CSV files to create your eCommerce analytics dashboard.")

In [ ]:
# 7. OPTIMIZED DATASETS FOR LOOKER STUDIO PERFORMANCE
print("\n⚡ Step 7: Creating optimized datasets for Looker Studio performance...")

def create_time_series_aggregations(sales_df, customer_df):
    """Create time-based aggregated datasets for faster Looker Studio performance"""
    
    # 7.1 YEARLY PERFORMANCE DATASET
    yearly_performance = sales_df.groupby('Year').agg({
        'ID': 'count',
        'Customer Email': 'nunique',
        'Grand Total (Base)': ['sum', 'mean'],
        'Discount Amount': 'sum',
        'Status': lambda x: (x == 'complete').sum()
    }).round(2)
    
    yearly_performance.columns = [
        'Total_Orders', 'Unique_Customers', 'Total_Revenue', 
        'Average_Order_Value', 'Total_Discounts', 'Completed_Orders'
    ]
    
    # Add YoY growth calculations
    yearly_performance['Revenue_Growth_Rate'] = yearly_performance['Total_Revenue'].pct_change() * 100
    yearly_performance['Customer_Growth_Rate'] = yearly_performance['Unique_Customers'].pct_change() * 100
    yearly_performance['Order_Growth_Rate'] = yearly_performance['Total_Orders'].pct_change() * 100
    
    # Calculate retention metrics per year
    for year in yearly_performance.index:
        year_customers = sales_df[sales_df['Year'] == year]['Customer Email'].unique()
        if year > yearly_performance.index.min():
            prev_year = year - 1
            prev_customers = sales_df[sales_df['Year'] == prev_year]['Customer Email'].unique()
            retained_customers = len(set(year_customers) & set(prev_customers))
            yearly_performance.loc[year, 'Retained_Customers'] = retained_customers
            yearly_performance.loc[year, 'Retention_Rate'] = (retained_customers / len(prev_customers) * 100) if len(prev_customers) > 0 else 0
    
    yearly_performance = yearly_performance.round(2).reset_index()
    
    # 7.2 MONTHLY TREND DATASET (Aggregated)
    monthly_trends = sales_df.groupby(['Year', 'Month']).agg({
        'ID': 'count',
        'Customer Email': 'nunique',
        'Grand Total (Base)': ['sum', 'mean'],
        'Discount Amount': 'sum'
    }).round(2)
    
    monthly_trends.columns = ['Orders', 'Customers', 'Revenue', 'AOV', 'Discounts']
    monthly_trends = monthly_trends.reset_index()
    monthly_trends['Month_Name'] = pd.to_datetime(monthly_trends[['Year', 'Month']].assign(day=1)).dt.strftime('%B')
    monthly_trends['Year_Month'] = monthly_trends['Year'].astype(str) + '-' + monthly_trends['Month'].astype(str).str.zfill(2)
    
    # 7.3 QUARTERLY PERFORMANCE DATASET
    quarterly_performance = sales_df.groupby(['Year', 'Quarter']).agg({
        'ID': 'count',
        'Customer Email': 'nunique',
        'Grand Total (Base)': ['sum', 'mean'],
        'Discount Amount': 'sum'
    }).round(2)
    
    quarterly_performance.columns = ['Orders', 'Customers', 'Revenue', 'AOV', 'Discounts']
    quarterly_performance = quarterly_performance.reset_index()
    quarterly_performance['Quarter_Label'] = 'Q' + quarterly_performance['Quarter'].astype(str) + ' ' + quarterly_performance['Year'].astype(str)
    
    return yearly_performance, monthly_trends, quarterly_performance

def create_customer_segments_analysis(customer_df):
    """Create customer segment analysis datasets"""
    
    # 7.4 CUSTOMER LIFETIME VALUE SEGMENTS
    clv_segments = customer_df.groupby('CLV_Segment').agg({
        'EMAIL': 'count',
        'Total_Revenue': ['sum', 'mean'],
        'Total_Orders': ['sum', 'mean'],
        'Average_Order_Value': 'mean',
        'Purchase_Frequency_Days': 'mean'
    }).round(2)
    
    clv_segments.columns = [
        'Customer_Count', 'Total_Segment_Revenue', 'Avg_Customer_Revenue',
        'Total_Segment_Orders', 'Avg_Customer_Orders', 'Avg_Order_Value', 'Avg_Purchase_Frequency'
    ]
    clv_segments = clv_segments.reset_index()
    
    # Calculate segment contribution percentages
    total_revenue = clv_segments['Total_Segment_Revenue'].sum()
    total_customers = clv_segments['Customer_Count'].sum()
    clv_segments['Revenue_Contribution_Pct'] = (clv_segments['Total_Segment_Revenue'] / total_revenue * 100).round(2)
    clv_segments['Customer_Contribution_Pct'] = (clv_segments['Customer_Count'] / total_customers * 100).round(2)
    
    # 7.5 REPEAT VS NEW CUSTOMER ANALYSIS
    repeat_analysis = customer_df.groupby('Repeat_Customer').agg({
        'EMAIL': 'count',
        'Total_Revenue': ['sum', 'mean'],
        'Total_Orders': ['sum', 'mean'],
        'Average_Order_Value': 'mean'
    }).round(2)
    
    repeat_analysis.columns = [
        'Customer_Count', 'Total_Revenue', 'Avg_Revenue_Per_Customer',
        'Total_Orders', 'Avg_Orders_Per_Customer', 'Average_Order_Value'
    ]
    repeat_analysis = repeat_analysis.reset_index()
    repeat_analysis['Customer_Type'] = repeat_analysis['Repeat_Customer'].map({True: 'Repeat Customer', False: 'One-time Customer'})
    
    return clv_segments, repeat_analysis

def create_cohort_summary(cohort_df):
    """Create summarized cohort analysis for dashboard"""
    
    # 7.6 COHORT RETENTION SUMMARY (Monthly cohorts with key periods)
    key_periods = [0, 1, 3, 6, 12]  # First purchase, 1 month, 3 months, 6 months, 1 year
    cohort_summary = cohort_df[cohort_df['Period_Number'].isin(key_periods)].copy()
    
    # Pivot for easier visualization
    cohort_pivot = cohort_summary.pivot(index='Cohort_Month', columns='Period_Number', values='Retention_Rate').reset_index()
    cohort_pivot.columns = ['Cohort_Month'] + [f'Month_{int(col)}' if col != 'Cohort_Month' else col for col in cohort_pivot.columns[1:]]
    
    # 7.7 COHORT SIZE AND ACQUISITION TRENDS
    cohort_acquisition = cohort_df[cohort_df['Period_Number'] == 0][['Cohort_Month', 'Cohort_Size']].copy()
    cohort_acquisition['Cohort_Month_Str'] = cohort_acquisition['Cohort_Month'].astype(str)
    cohort_acquisition['Year'] = cohort_acquisition['Cohort_Month'].dt.year
    cohort_acquisition['Month'] = cohort_acquisition['Cohort_Month'].dt.month
    
    return cohort_pivot, cohort_acquisition

def create_email_subscriber_analysis(retention_df):
    """Create email subscriber analysis datasets"""
    
    # 7.8 EMAIL SUBSCRIBER TRENDS
    newsletter_analysis = retention_df.groupby('NEWSLETTER_FREQUENCY').agg({
        'EMAIL': 'count',
        'TOTAL_ORDER_AMOUNT': ['sum', 'mean', 'count']
    }).round(2)
    
    newsletter_analysis.columns = ['Subscriber_Count', 'Total_Revenue', 'Avg_Revenue_Per_Subscriber', 'Active_Buyers']
    newsletter_analysis = newsletter_analysis.reset_index()
    newsletter_analysis['Conversion_Rate'] = (newsletter_analysis['Active_Buyers'] / newsletter_analysis['Subscriber_Count'] * 100).round(2)
    
    # 7.9 DEMOGRAPHIC ANALYSIS
    demographic_analysis = retention_df.groupby('GENDER').agg({
        'EMAIL': 'count',
        'TOTAL_ORDER_AMOUNT': ['sum', 'mean']
    }).round(2)
    
    demographic_analysis.columns = ['Customer_Count', 'Total_Revenue', 'Avg_Revenue_Per_Customer']
    demographic_analysis = demographic_analysis.reset_index()
    
    # Regional analysis
    regional_analysis = retention_df.groupby('REGION').agg({
        'EMAIL': 'count',
        'TOTAL_ORDER_AMOUNT': ['sum', 'mean']
    }).round(2)
    
    regional_analysis.columns = ['Customer_Count', 'Total_Revenue', 'Avg_Revenue_Per_Customer']
    regional_analysis = regional_analysis.reset_index()
    regional_analysis = regional_analysis.sort_values('Total_Revenue', ascending=False).head(20)  # Top 20 regions
    
    return newsletter_analysis, demographic_analysis, regional_analysis

def create_dashboard_kpis(sales_df, customer_df):
    """Create KPI summary datasets for different time periods"""
    
    # 7.10 MONTHLY KPI TRENDS
    monthly_kpis = []
    
    for month_year in sales_df['Month_Year'].unique():
        month_data = sales_df[sales_df['Month_Year'] == month_year]
        month_customers = customer_df[customer_df['EMAIL'].isin(month_data['Customer Email'].unique())]
        
        total_customers = month_data['Customer Email'].nunique()
        repeat_customers = len(month_customers[month_customers['Repeat_Customer']])
        
        kpi_row = {
            'Month_Year': month_year,
            'Total_Orders': len(month_data),
            'Total_Revenue': month_data['Grand Total (Base)'].sum(),
            'Total_Customers': total_customers,
            'New_Customers': total_customers - repeat_customers,
            'Repeat_Customers': repeat_customers,
            'AOV': month_data['Grand Total (Base)'].mean(),
            'Retention_Rate': (repeat_customers / total_customers * 100) if total_customers > 0 else 0
        }
        monthly_kpis.append(kpi_row)
    
    monthly_kpis_df = pd.DataFrame(monthly_kpis).round(2)
    monthly_kpis_df = monthly_kpis_df.sort_values('Month_Year')
    
    # 7.11 YEARLY KPI SUMMARY
    yearly_kpis = []
    
    for year in sales_df['Year'].unique():
        year_data = sales_df[sales_df['Year'] == year]
        year_customers = customer_df[customer_df['EMAIL'].isin(year_data['Customer Email'].unique())]
        
        total_customers = year_data['Customer Email'].nunique()
        repeat_customers = len(year_customers[year_customers['Repeat_Customer']])
        
        kpi_row = {
            'Year': year,
            'Total_Orders': len(year_data),
            'Total_Revenue': year_data['Grand Total (Base)'].sum(),
            'Total_Customers': total_customers,
            'New_Customers': total_customers - repeat_customers,
            'Repeat_Customers': repeat_customers,
            'AOV': year_data['Grand Total (Base)'].mean(),
            'Retention_Rate': (repeat_customers / total_customers * 100) if total_customers > 0 else 0
        }
        yearly_kpis.append(kpi_row)
    
    yearly_kpis_df = pd.DataFrame(yearly_kpis).round(2)
    yearly_kpis_df = yearly_kpis_df.sort_values('Year')
    
    return monthly_kpis_df, yearly_kpis_df

In [ ]:
# Generate all optimized datasets
print("🔄 Creating yearly performance dataset...")
yearly_perf, monthly_trends, quarterly_perf = create_time_series_aggregations(sales_clean, customer_analytics)

print(f"Shape: {yearly_perf.shape}")
yearly_perf.head(3)

In [ ]:
print(f"Shape: {monthly_trends.shape}")

monthly_trends.head(3)

In [ ]:
print(f"Shape: {quarterly_perf.shape}")
quarterly_perf.head(3)

In [ ]:
print("🔄 Creating customer segment analysis...")
clv_segments, repeat_analysis = create_customer_segments_analysis(customer_analytics)

print(f"Shape: {clv_segments.shape}")
clv_segments.head(3)

In [ ]:
print(f"Shape: {repeat_analysis.shape}")
repeat_analysis.head()

In [ ]:
print("🔄 Creating cohort summaries...")
cohort_pivot, cohort_acquisition = create_cohort_summary(cohort_analysis)

print(f"Shape: {cohort_pivot.shape}")
cohort_pivot.head(3)

In [ ]:
print(f"Shape: {cohort_acquisition.shape}")
cohort_acquisition.head(3)

In [ ]:
print("🔄 Creating email subscriber analysis...")
newsletter_analysis, demographic_analysis, regional_analysis = create_email_subscriber_analysis(retention_clean)

print(f"Shape: {newsletter_analysis.shape}")
newsletter_analysis.head(3)

In [ ]:
print(f"Shape: {demographic_analysis.shape}")
demographic_analysis.head(3)

In [ ]:
print(f"Shape: {regional_analysis.shape}")
regional_analysis.head(3)

In [ ]:
print("🔄 Creating dashboard KPIs...")
monthly_kpis, yearly_kpis = create_dashboard_kpis(sales_clean, customer_analytics)

print(f"Shape: {monthly_kpis.shape}")
monthly_kpis.head(3)

In [ ]:
print(f"Shape: {yearly_kpis.shape}")
yearly_kpis.head(3)

In [ ]:
# 8. SAMPLE DATASETS FOR TESTING
print("\n🧪 Creating sample datasets for testing...")

# Create smaller sample datasets for initial dashboard development
sample_datasets = {
    'sample_sales_data.csv': sales_clean.sample(n=min(10000, len(sales_clean))),
    'sample_customer_analytics.csv': customer_analytics.sample(n=min(5000, len(customer_analytics))),
    'sample_monthly_trends.csv': monthly_trends
}

for filename, dataset in sample_datasets.items():
    dataset.to_csv(filename, index=False)
    print(f"✅ Saved: {filename} ({dataset.shape})")

# 9. GENERATE COMPREHENSIVE DATA DICTIONARY
print("\n📋 Step 8: Generating comprehensive data dictionary...")

data_dictionary = {
    'File': [],
    'Column': [],
    'Description': [],
    'Data_Type': [],
    'Use_Case': []
}

# Comprehensive file descriptions
file_descriptions = {
    'yearly_performance.csv': {
        'Year': ('Year', 'int', 'Year-over-year analysis, growth trends'),
        'Total_Orders': ('Total number of orders in year', 'int', 'Annual performance tracking'),
        'Total_Revenue': ('Total revenue generated in year', 'float', 'Revenue trend analysis'),
        'Revenue_Growth_Rate': ('Year-over-year revenue growth percentage', 'float', 'Growth rate visualization'),
        'Retention_Rate': ('Customer retention rate for the year', 'float', 'Retention trend analysis')
    },
    'monthly_trends.csv': {
        'Year': ('Year', 'int', 'Monthly trend analysis'),
        'Month': ('Month number (1-12)', 'int', 'Monthly trend analysis'),
        'Orders': ('Number of orders in month', 'int', 'Monthly performance tracking'),
        'Revenue': ('Total revenue in month', 'float', 'Monthly revenue trends'),
        'Customers': ('Unique customers in month', 'int', 'Customer acquisition trends')
    },
    'customer_segments_clv.csv': {
        'CLV_Segment': ('Customer lifetime value segment', 'str', 'Customer segmentation analysis'),
        'Customer_Count': ('Number of customers in segment', 'int', 'Segment size analysis'),
        'Total_Segment_Revenue': ('Total revenue from segment', 'float', 'Segment contribution analysis'),
        'Revenue_Contribution_Pct': ('Percentage of total revenue from segment', 'float', 'Segment importance metrics')
    },
    'cohort_retention_summary.csv': {
        'Cohort_Month': ('Month when customers made first purchase', 'str', 'Cohort analysis'),
        'Month_0': ('Retention rate at first purchase (100%)', 'float', 'Cohort retention tracking'),
        'Month_1': ('Retention rate after 1 month', 'float', 'Cohort retention tracking'),
        'Month_3': ('Retention rate after 3 months', 'float', 'Cohort retention tracking'),
        'Month_6': ('Retention rate after 6 months', 'float', 'Cohort retention tracking'),
        'Month_12': ('Retention rate after 12 months', 'float', 'Cohort retention tracking')
    },
    'monthly_kpi_trends.csv': {
        'Month_Year': ('Month-Year period', 'str', 'KPI trend analysis'),
        'Total_Orders': ('Orders in month', 'int', 'Performance monitoring'),
        'Total_Revenue': ('Revenue in month', 'float', 'Revenue tracking'),
        'Retention_Rate': ('Customer retention rate for month', 'float', 'Retention monitoring'),
        'AOV': ('Average order value for month', 'float', 'Order value trends')
    }
}

for file, columns in file_descriptions.items():
    for col, (desc, dtype, use_case) in columns.items():
        data_dictionary['File'].append(file)
        data_dictionary['Column'].append(col)
        data_dictionary['Description'].append(desc)
        data_dictionary['Data_Type'].append(dtype)
        data_dictionary['Use_Case'].append(use_case)

pd.DataFrame(data_dictionary).to_csv('comprehensive_data_dictionary.csv', index=False)

print("\n🎉 Optimized data processing completed successfully!")
print("="*50)
print("📁 OPTIMIZED FILES FOR LOOKER STUDIO (Small & Fast):")
for filename in optimized_datasets.keys():
    print(f"   • {filename}")

print("\n📁 SAMPLE FILES FOR TESTING:")
for filename in sample_datasets.keys():
    print(f"   • {filename}")

print("\n📁 ORIGINAL COMPLETE FILES (Backup):")
for filename in datasets.keys():
    print(f"   • {filename}")


In [ ]:
# Save optimized datasets
optimized_datasets = {
    # Time-based analysis
    'yearly_performance.csv': yearly_perf,
    'monthly_trends.csv': monthly_trends,
    'quarterly_performance.csv': quarterly_perf,
    
    # Customer analysis
    'customer_segments_clv.csv': clv_segments,
    'repeat_vs_new_customers.csv': repeat_analysis,
    
    # Cohort analysis
    'cohort_retention_summary.csv': cohort_pivot,
    'cohort_acquisition_trends.csv': cohort_acquisition,
    
    # Email & Demographics
    'newsletter_subscriber_analysis.csv': newsletter_analysis,
    'demographic_analysis.csv': demographic_analysis,
    'regional_performance.csv': regional_analysis,
    
    # KPI Dashboards
    'monthly_kpi_trends.csv': monthly_kpis,
    'yearly_kpi_summary.csv': yearly_kpis
}

print("\n💾 Saving optimized datasets for Looker Studio...")
for filename, dataset in optimized_datasets.items():
    dataset.to_csv(filename, index=False)
    print(f"✅ Saved: {filename} ({dataset.shape})")


print("\n💾 Save done")

In [ ]:
print("\n📊 DASHBOARD RECOMMENDATIONS:")
print("   📈 Use 'yearly_performance.csv' for year-over-year analysis")
print("   📅 Use 'monthly_trends.csv' for monthly trend charts")
print("   👥 Use 'customer_segments_clv.csv' for customer segmentation")
print("   🔄 Use 'cohort_retention_summary.csv' for cohort analysis")
print("   📧 Use 'newsletter_subscriber_analysis.csv' for email metrics")
print("   📋 Use 'monthly_kpi_trends.csv' for KPI dashboards")

print(f"\n📊 Key Statistics:")
print(f"   • Total Orders: {len(sales_clean):,}")
print(f"   • Total Customers: {sales_clean['Customer Email'].nunique():,}")
print(f"   • Date Range: {sales_clean['Purchase Date'].min().strftime('%Y-%m-%d')} to {sales_clean['Purchase Date'].max().strftime('%Y-%m-%d')}")
print(f"   • Total Revenue: ${sales_clean['Grand Total (Base)'].sum():,.2f}")

print("\n🚀 READY FOR LOOKER STUDIO!")
print("Start with the optimized datasets for better dashboard performance!")

In [ ]:
# Function to summarize each DataFrame
def summarize_df(df, name):
    print(f"\n📄 SUMMARY FOR: {name}\n")
    print("\n Info:")
    print(df.info())
    print("First 5 Rows:")

In [ ]:
summarize_df(yearly_perf, "yearly_performance")
yearly_perf.head()

In [ ]:
summarize_df(monthly_trends, "monthly_trends")
monthly_trends.head()

In [ ]:
summarize_df(clv_segments, "clv_segments")
clv_segments.head()

In [ ]:
summarize_df(cohort_pivot, "cohort_pivot")
cohort_pivot.head()

In [ ]:
summarize_df(newsletter_analysis, "newsletter_analysis")
newsletter_analysis.head()

In [ ]:
summarize_df(monthly_kpis, "monthly_kpis")
monthly_kpis.head()

## Finding Monthly top 10 SKUs 

In [ ]:
# Read the CSV files (update paths as needed)
file1 = "/kaggle/input/crm-data-analysis/mehedi-export.csv"

# Load the data into DataFrames
export_df = pd.read_csv(file1, encoding='utf-8', low_memory=False)

print(f"✅ Dataset shape: {export_df.shape}")

In [ ]:
export_df.head(5)

In [ ]:
export_df.info()

In [ ]:
import pandas as pd

# Step 1: Load the data
file_path = "/kaggle/input/crm-data-analysis/mehedi-export.csv"
export_df = pd.read_csv(file_path, encoding='utf-8', low_memory=False)

# Step 2: Convert 'created_at' to datetime and extract 'YearMonth'
export_df['created_at'] = pd.to_datetime(export_df['created_at'], errors='coerce')
export_df['YearMonth'] = export_df['created_at'].dt.to_period('M').astype(str)

# Step 3: Drop unnecessary columns
columns_to_keep = ['customer_email', 'created_at', 'YearMonth', 'sku_list']
export_df = export_df[columns_to_keep]

# Step 4: Clean sku_list and explode rows
export_df['sku_list'] = export_df['sku_list'].str.replace(r'\s+', '', regex=True)
export_df['sku_list'] = export_df['sku_list'].str.split(',')
df_exploded = export_df.explode('sku_list')

# Remove any empty or NaN SKUs after splitting
df_exploded = df_exploded[df_exploded['sku_list'].notna() & (df_exploded['sku_list'] != '')]

# Step 5: Group by YearMonth and SKU to count purchases
sku_counts = (
    df_exploded
    .groupby(['YearMonth', 'sku_list'])
    .size()
    .reset_index(name='purchase_count')
)

In [ ]:
export_df.head(5)

In [ ]:
sku_counts

In [ ]:
# Step 6: Get Top 10 SKUs per month
top10_skus_monthly = (
    sku_counts
    .sort_values(['YearMonth', 'purchase_count'], ascending=[True, False])
    .groupby('YearMonth')
    .head(10)
    .reset_index(drop=True)
)

In [ ]:
# Optional: Save the result to CSV for Looker Studio upload
output_file = "top10_skus_monthly.csv"
top10_skus_monthly.to_csv(output_file, index=False)

print(f"✅ Top 10 SKUs per month saved to: {output_file}")


In [ ]:
export_df.head(5)

In [ ]:
# Sorted list of unique SKUs
unique_skus_sorted = sorted(df_exploded['sku_list'].dropna().unique())
print(unique_skus_sorted[:6])

In [ ]:
# Ensure all SKUs are strings
df_exploded['sku_str'] = df_exploded['sku_list'].astype(str)

# Filter rows where SKU is NOT a number
non_numeric_skus = df_exploded[~df_exploded['sku_str'].str.isdigit()]

# Show unique non-numeric SKUs
print(non_numeric_skus['sku_list'].unique())

In [ ]:
non_numeric_skus = [sku for sku in unique_skus_sorted if not str(sku).isdigit()]
print(non_numeric_skus)

# Top 15 SKUs per year

In [ ]:
# # Step 1: Load the data
# file_path = "/kaggle/input/crm-data-analysis/mehedi-export.csv"
# export_df = pd.read_csv(file_path, encoding='utf-8', low_memory=False)

# export_df.head(5)

In [ ]:
# # Step 2: Convert 'created_at' to datetime and extract 'Year'
# export_df['created_at'] = pd.to_datetime(export_df['created_at'], errors='coerce')
# export_df['Year'] = export_df['created_at'].dt.year

# # Step 3: Keep only necessary columns
# columns_to_keep = ['customer_email', 'created_at', 'Year', 'sku_list']
# export_df = export_df[columns_to_keep]

# # Step 4: Clean and explode SKU list
# export_df['sku_list'] = export_df['sku_list'].astype(str).str.replace(r'\s+', '', regex=True)
# export_df['sku_list'] = export_df['sku_list'].str.split(',')
# df_exploded = export_df.explode('sku_list')

# # Filter out empty or NaN SKUs
# df_exploded = df_exploded[df_exploded['sku_list'].notna() & (df_exploded['sku_list'] != '')]

# # ✅ Filter only numeric SKUs
# df_exploded = df_exploded[df_exploded['sku_list'].str.isdigit()]

# # Step 5: Count purchases per SKU per year
# sku_counts_yearly = (
#     df_exploded
#     .groupby(['Year', 'sku_list'])
#     .size()
#     .reset_index(name='purchase_count')
# )

# # Step 6: Get Top 15 SKUs per year
# top15_skus_yearly = (
#     sku_counts_yearly
#     .sort_values(['Year', 'purchase_count'], ascending=[True, False])
#     .groupby('Year')
#     .head(15)
#     .reset_index(drop=True)
# )

# # Step 7: Get repurchase rate for each SKU in top15_skus_yearly
# # Count purchases per customer per SKU per year
# sku_customer_counts = (
#     df_exploded
#     .groupby(['Year', 'sku_list', 'customer_email'])
#     .size()
#     .reset_index(name='purchase_count')
# )

# # Add repurchase flag
# sku_customer_counts['repurchased'] = sku_customer_counts['purchase_count'] >= 2

# # Filter to only SKUs in the top 15 for that year
# top_skus_set = set(zip(top15_skus_yearly['Year'], top15_skus_yearly['sku_list']))
# sku_customer_counts = sku_customer_counts[
#     sku_customer_counts.apply(lambda row: (row['Year'], row['sku_list']) in top_skus_set, axis=1)
# ]

# # Compute repurchase rate (as percentage)
# repurchase_rate = (
#     sku_customer_counts
#     .groupby(['Year', 'sku_list'])
#     .agg(
#         total_customers=('customer_email', 'nunique'),
#         repurchasers=('repurchased', 'sum')
#     )
#     .reset_index()
# )

# repurchase_rate['repurchase_rate'] = (
#     (repurchase_rate['repurchasers'] / repurchase_rate['total_customers']) * 100
# ).round(2)

# # Step 8: Merge top 15 SKUs with their repurchase rate
# final_result = pd.merge(top15_skus_yearly, repurchase_rate, on=['Year', 'sku_list'])

# # Final output preview
# print("\n✅ Top 15 SKUs with Repurchase Rate per Year:")
# final_result.head(10)

In [ ]:
# final_result['sku_list']

In [ ]:
# # Step 9: Save to CSV
# final_result.to_csv("top15_skus_repurchase_rate_yearly.csv", index=False)
# print("\n📁 File 'top15_skus_repurchase_rate_yearly.csv' saved successfully.")

# Product Catalog with SKU ID (Yearly 2023-2025) (Not fiscal)

In [ ]:
import pandas as pd

# Load both CSV files
purchase_data_path = "/kaggle/input/crm-data-analysis/mehedi-export.csv"
product_catalog_path = "/kaggle/input/crm-data-analysis/product catalog - Sheet1.csv"

purchase_df = pd.read_csv(purchase_data_path, encoding='utf-8', low_memory=False)

print(purchase_df.info())
purchase_df.head(5)

In [ ]:
product_df = pd.read_csv(product_catalog_path, encoding='utf-8', low_memory=False)
product_df.head(5)

In [ ]:
product_df.info()

In [ ]:
# --- Clean Product Catalog ---
product_df.columns = product_df.columns.str.strip()
product_df = product_df.rename(columns={
    'Model Name - !!!DO NOT CHANGE THE ORDER!!!': 'Model_Name',
    'SKU': 'SKU',
    'Typ I': 'Type_1'
})

# Remove accessories from 'Type_1'
product_df['Type_1'] = product_df['Type_1'].astype(str).str.strip()
product_df = product_df[product_df['Type_1'].str.lower() != 'accessories']

# Clean Model_Name and SKU
product_df['Model_Name'] = product_df['Model_Name'].astype(str).str.strip()
product_df['SKU'] = product_df['SKU'].astype(str).str.strip()

# --- Clean and Prepare Purchase Data ---
purchase_df['created_at'] = pd.to_datetime(purchase_df['created_at'], errors='coerce')
purchase_df['Year'] = purchase_df['created_at'].dt.year

columns_to_keep = ['customer_email', 'created_at', 'Year', 'sku_list']
purchase_df = purchase_df[columns_to_keep]

purchase_df['sku_list'] = purchase_df['sku_list'].astype(str).str.replace(r'\s+', '', regex=True)
purchase_df['sku_list'] = purchase_df['sku_list'].str.split(',')

df_exploded = purchase_df.explode('sku_list')

# Remove empty or NaN SKUs
df_exploded = df_exploded[df_exploded['sku_list'].notna() & (df_exploded['sku_list'] != '')]

# --- Merge with Product Catalog ---
# Convert SKU column to string before merge
df_exploded['sku_list'] = df_exploded['sku_list'].astype(str)
merged_df = pd.merge(df_exploded, product_df, how='inner', left_on='sku_list', right_on='SKU')

# ✅ Filter only numerical SKU IDs after merge
merged_df = merged_df[merged_df['sku_list'].str.isdigit()]

merged_df.head(5)

In [ ]:
merged_df.info()

In [ ]:
merged_df['sku_list'].info()

In [ ]:
# Step 1: Count purchases per SKU per year
sku_counts_yearly = (
    merged_df
    .groupby(['Year', 'sku_list'])
    .size()
    .reset_index(name='purchase_count')
)

# Step 2: Top 15 SKUs per year
top15_skus_yearly = (
    sku_counts_yearly
    .sort_values(['Year', 'purchase_count'], ascending=[True, False])
    .groupby('Year')
    .head(15)
    .reset_index(drop=True)
)

# Step 3: Repurchase Calculation
sku_customer_counts = (
    merged_df
    .groupby(['Year', 'sku_list', 'customer_email'])
    .size()
    .reset_index(name='purchase_count')
)

sku_customer_counts['repurchased'] = sku_customer_counts['purchase_count'] >= 2

# Filter only SKUs in top 15
top_skus_set = set(zip(top15_skus_yearly['Year'], top15_skus_yearly['sku_list']))
sku_customer_counts = sku_customer_counts[
    sku_customer_counts.apply(lambda row: (row['Year'], row['sku_list']) in top_skus_set, axis=1)
]

# Compute repurchase rate
repurchase_df = (
    sku_customer_counts
    .groupby(['Year', 'sku_list'])
    .agg(
        total_customers=('customer_email', 'nunique'),
        repurchasers=('repurchased', 'sum')
    )
    .reset_index()
)

repurchase_df['repurchase_rate'] = (
    (repurchase_df['repurchasers'] / repurchase_df['total_customers']) * 100
).round(2)

# Step 4: Final Merge for Report
final_df = pd.merge(top15_skus_yearly, repurchase_df, on=['Year', 'sku_list'])

# Merge with product info (Model Name, Type_1)
final_df = pd.merge(final_df, product_df, how='left', left_on='sku_list', right_on='SKU')

# Final columns
final_df = final_df[['Year', 'Model_Name', 'SKU', 'Type_1', 'purchase_count', 'total_customers', 'repurchasers', 'repurchase_rate']]


final_df.head(5)

In [ ]:
# Save to CSV
final_df.to_csv("Top15_SKUs_model_repurchase", index=False)
print("✅ File saved as 'Top15_SKUs_model_repurchase'")

# Product Catalog with SKU ID (Yearly April 2023 - March 2025) (Fiscal)

In [ ]:
purchase_df = pd.read_csv(purchase_data_path, encoding='utf-8', low_memory=False)
product_df = pd.read_csv(product_catalog_path, encoding='utf-8', low_memory=False)

In [ ]:
# First, ensure created_at is in datetime format (you've already done this in your script)
purchase_df['created_at'] = pd.to_datetime(purchase_df['created_at'], errors='coerce')

# Get the earliest and latest date
min_date = purchase_df['created_at'].min()
max_date = purchase_df['created_at'].max()

# Summary of the created_at column
summary = purchase_df['created_at'].describe()

print(f"📅 First order date: {min_date}")
print(f"📅 Last order date: {max_date}")

In [ ]:
# Clean Product Catalog
product_df.columns = product_df.columns.str.strip()
product_df = product_df.rename(columns={
    'Model Name - !!!DO NOT CHANGE THE ORDER!!!': 'Model_Name',
    'SKU': 'SKU',
    'Typ I': 'Type_1'
})
product_df['Type_1'] = product_df['Type_1'].astype(str).str.strip()
product_df = product_df[product_df['Type_1'].str.lower() != 'accessories']
product_df['Model_Name'] = product_df['Model_Name'].astype(str).str.strip()
product_df['SKU'] = product_df['SKU'].astype(str).str.strip()

# Clean and Prepare Purchase Data
purchase_df['created_at'] = pd.to_datetime(purchase_df['created_at'], errors='coerce')

# 🎯 Create Fiscal Year column
def get_fiscal_year(date):
    if pd.isnull(date):
        return None
    year = date.year
    if date.month < 4:
        return f"{year-1}-{year}"
    else:
        return f"{year}-{year+1}"

purchase_df['Fiscal_Year'] = purchase_df['created_at'].apply(get_fiscal_year)

# Keep required columns
columns_to_keep = ['customer_email', 'created_at', 'Fiscal_Year', 'sku_list']
purchase_df = purchase_df[columns_to_keep]

purchase_df['sku_list'] = purchase_df['sku_list'].astype(str).str.replace(r'\s+', '', regex=True)
purchase_df['sku_list'] = purchase_df['sku_list'].str.split(',')

df_exploded = purchase_df.explode('sku_list')
df_exploded = df_exploded[df_exploded['sku_list'].notna() & (df_exploded['sku_list'] != '')]

# Merge with Product Catalog
df_exploded['sku_list'] = df_exploded['sku_list'].astype(str)
merged_df = pd.merge(df_exploded, product_df, how='inner', left_on='sku_list', right_on='SKU')
merged_df = merged_df[merged_df['sku_list'].str.isdigit()]

# Step 1: Count purchases per SKU per Fiscal Year
sku_counts_yearly = (
    merged_df
    .groupby(['Fiscal_Year', 'sku_list'])
    .size()
    .reset_index(name='purchase_count')
)

# Step 2: Top 15 SKUs per Fiscal Year
top15_skus_yearly = (
    sku_counts_yearly
    .sort_values(['Fiscal_Year', 'purchase_count'], ascending=[True, False])
    .groupby('Fiscal_Year')
    .head(15)
    .reset_index(drop=True)
)

# Step 3: Repurchase Calculation
sku_customer_counts = (
    merged_df
    .groupby(['Fiscal_Year', 'sku_list', 'customer_email'])
    .size()
    .reset_index(name='purchase_count')
)

sku_customer_counts['repurchased'] = sku_customer_counts['purchase_count'] >= 2

# Filter only SKUs in top 15
top_skus_set = set(zip(top15_skus_yearly['Fiscal_Year'], top15_skus_yearly['sku_list']))
sku_customer_counts = sku_customer_counts[
    sku_customer_counts.apply(lambda row: (row['Fiscal_Year'], row['sku_list']) in top_skus_set, axis=1)
]

# Compute repurchase rate
repurchase_df = (
    sku_customer_counts
    .groupby(['Fiscal_Year', 'sku_list'])
    .agg(
        total_customers=('customer_email', 'nunique'),
        repurchasers=('repurchased', 'sum')
    )
    .reset_index()
)

repurchase_df['repurchase_rate'] = (
    (repurchase_df['repurchasers'] / repurchase_df['total_customers']) * 100
).round(2)

# Step 4: Final Merge for Report
final_df = pd.merge(top15_skus_yearly, repurchase_df, on=['Fiscal_Year', 'sku_list'])
final_df = pd.merge(final_df, product_df, how='left', left_on='sku_list', right_on='SKU')

# Final columns
final_df = final_df[['Fiscal_Year', 'Model_Name', 'SKU', 'Type_1', 'purchase_count', 'total_customers', 'repurchasers', 'repurchase_rate']]

# Save to CSV
final_df.to_csv("Top15_SKUs_model_repurchase_by_fiscal_year.csv", index=False)
print("✅ File saved as 'Top15_SKUs_model_repurchase_by_fiscal_year.csv'")

# New & Repurchase Customer

In [ ]:
import pandas as pd

# Load purchase data
purchase_data_path = "/kaggle/input/crm-data-analysis/mehedi-export.csv"
purchase_df = pd.read_csv(purchase_data_path, encoding='utf-8', low_memory=False)

# Step 1: Preprocess dates and emails
purchase_df['created_at'] = pd.to_datetime(purchase_df['created_at'], errors='coerce')
purchase_df['Year'] = purchase_df['created_at'].dt.year
purchase_df['customer_email'] = purchase_df['customer_email'].str.strip().str.lower()

# Step 2: Identify New Customers per Year
new_customers_by_year = []
all_previous_customers = set()

for year in sorted(purchase_df['Year'].dropna().unique()):
    current_year_df = purchase_df[purchase_df['Year'] == year]
    current_customers = set(current_year_df['customer_email'].dropna().unique())

    new_customers = current_customers - all_previous_customers

    new_customers_by_year.extend([(year, email) for email in new_customers])

    # Update for next loop
    all_previous_customers.update(current_customers)

new_customers_df = pd.DataFrame(new_customers_by_year, columns=['Year', 'customer_email'])

new_customers_df.head(5)

In [ ]:
# Step 3: Identify Repurchasing Customers (multiple orders in same year)
repurchase_df = (
    purchase_df.groupby(['Year', 'customer_email'])
    .size()
    .reset_index(name='purchase_count')
)

repurchase_df = repurchase_df[repurchase_df['purchase_count'] >= 2]
repurchasing_customers_df = repurchase_df[['Year', 'customer_email']]


repurchasing_customers_df.head(5)

In [ ]:
# Step 4: Save both lists into a single Excel file (with two sheets)
output_filename = "customer_summary.csv"

# Saving as Excel for multiple sheets
with pd.ExcelWriter("customer_summary.xlsx") as writer:
    new_customers_df.to_excel(writer, sheet_name='New_Customers', index=False)
    repurchasing_customers_df.to_excel(writer, sheet_name='Repurchasers', index=False)

print("✅ Saved to 'customer_summary.xlsx' with two sheets: 'New_Customers' and 'Repurchasers'")